# --- Data Cleaning and Preprocessing ---

In [1]:
import pandas as pd 
import numpy as np
import os 

#file paths 

raw_path = r"C:/Users/ronv1/Desktop/Automative_Warranty_Analytics/data/raw"
clean_path = r"C:/Users/ronv1/Desktop/Automative_Warranty_Analytics/data/cleaned"

os.makedirs(clean_path, exist_ok = True)

In [2]:
# load data 

vehicles = pd.read_csv(os.path.join(raw_path,"vehicles_raw.csv"))
service_records = pd.read_csv(os.path.join(raw_path, "service_records_raw.csv"))
warranty_claims = pd.read_csv(os.path.join(raw_path,"warranty_claims_raw.csv"))
sensor_data = pd.read_csv(os.path.join(raw_path, "sensor_data_raw.csv"))


# shape_check
print(vehicles.shape, service_records.shape, warranty_claims.shape, sensor_data.shape)


(10000, 5) (60000, 6) (20000, 6) (100000, 6)


In [8]:
# data preview

print("Vehicles data: ")
print(vehicles.head(10),"\n")

print("Service Records: ")
print(service_records.head(10), "\n")

print("Warranty claims: ")
print(warranty_claims.head(10), "\n")

print("Sensor data: ")
print(sensor_data.head(10), "\n")


Vehicles data: 
  vehicle_id     model  year   mileage      region
0     V10000   AutoMax  2023  110212.0        West
1     V10001   AutoMax  2022   97716.0       South
2     V10002   UrbanGo  2016    9132.0        East
3     V10003    SpeedX  2022    8325.0       North
4     V10004  EcoDrive  2019   28748.0       SOUTH
5     V10005  EcoDrive  2020   52545.0  south zone
6     V10006   UrbanGo  2023   38790.0  south zone
7     V10007   AutoMax  2018   40748.0       North
8     V10008   AutoMax  2019  116331.0  south zone
9     V10009   UrbanGo  2018   14097.0  south zone 

Service Records: 
  service_id vehicle_id service_date component_replaced   cost   issue_type
0     S20000     V12551   2023-01-11            Battery  13529  Wear & Tear
1     S20001     V13012   2023-06-12         Suspension   3495  Wear & Tear
2     S20002     V16549   2024-02-06             Engine  14162    Discharge
3     S20003     V17432   2024-07-11       Transmission   1256      Leakage
4     S20004     V10933

In [12]:
# dictionary for looping through all datasets

datasets = {
    "Vehicles" : vehicles,
    "Service Records" : service_records,
    "Warranty Claims" : warranty_claims,
    "Sensor data" : sensor_data 
}

# display info and missing values

for name, df in datasets.items():
    print(f"{name} Info: ")
    print(df.info(), "\n")
    print(f"Missing values in {name}: ")
    print(df.isnull().sum(), "\n")
        

Vehicles Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vehicle_id  10000 non-null  object 
 1   model       10000 non-null  object 
 2   year        10000 non-null  int64  
 3   mileage     9525 non-null   float64
 4   region      10000 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 390.8+ KB
None 

Missing values in Vehicles: 
vehicle_id      0
model           0
year            0
mileage       475
region          0
dtype: int64 

Service Records Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   service_id          60000 non-null  object
 1   vehicle_id          60000 non-null  object
 2   service_date        60000 non-null  object
 3   component

# Clean vehicles table 

In [19]:
# fix model inconsistencies

vehicles["model"] = vehicles["model"].str.strip().replace({
    "AutoMa" : "AutoMax",
    "EcoDriv" : "EcoDrive",
    "EcoDrivX" : "EcoDrive",
    "SpeedX" : "Speed",
    "TurboPr" : "TurboPro",
    "TurboPrX" : "TurboPro",
    "UrbanG" : "UrbanGo",
    "UrbanGX" : "UrbanGo"
})

# fill missing value with milage median

vehicles["mileage"] = vehicles["mileage"].fillna(vehicles["mileage"].median())

# fix region inconsistencies

vehicles["region"] = vehicles["region"].str.strip().replace({
    "south zone" : "South",
    "SOUTH" : "South",
    "south" : "South"
})

# drop duplicate values

vehicles.drop_duplicates(inplace = True)

# display cleaned data
print("Vehicles table cleaned ")
print(vehicles.head(), "\n")
print(vehicles.info())

Vehicles table cleaned 
  vehicle_id     model  year   mileage region
0     V10000   AutoMax  2023  110212.0   West
1     V10001   AutoMax  2022   97716.0  South
2     V10002   UrbanGo  2016    9132.0   East
3     V10003     Speed  2022    8325.0  North
4     V10004  EcoDrive  2019   28748.0  South 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vehicle_id  10000 non-null  object 
 1   model       10000 non-null  object 
 2   year        10000 non-null  int64  
 3   mileage     10000 non-null  float64
 4   region      10000 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 390.8+ KB
None


# Clean service records table 

In [20]:
# convert service date into datetime dtype

service_records["service_date"] = pd.to_datetime(service_records["service_date"], errors="coerce")

# fill missing componets with "Unknown"

service_records["component_replaced"] = service_records["component_replaced"].fillna("Unknown")

# replace negative cost with Nan

service_records.loc[service_records["cost"] < 0, "cost"] = np.nan

# drop duplicates

service_records.drop_duplicates(inplace = True)

In [33]:
# fill missing cost

service_records["cost"] = service_records["cost"].fillna(0)

print("Service records table cleaned ")
print(service_records.head(), "\n")
print(service_records.info())

Service records table cleaned 
  service_id vehicle_id service_date component_replaced     cost   issue_type
0     S20000     V12551   2023-01-11            Battery  13529.0  Wear & Tear
1     S20001     V13012   2023-06-12         Suspension   3495.0  Wear & Tear
2     S20002     V16549   2024-02-06             Engine  14162.0    Discharge
3     S20003     V17432   2024-07-11       Transmission   1256.0      Leakage
4     S20004     V10933   2024-06-27       Transmission   1708.0  Malfunction 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   service_id          60000 non-null  object        
 1   vehicle_id          60000 non-null  object        
 2   service_date        60000 non-null  datetime64[ns]
 3   component_replaced  60000 non-null  object        
 4   cost                60000 non-null  float64       

# Clean warranty claims table

In [23]:
# convert claim date into datetime dtype

warranty_claims["claim_date"] = pd.to_datetime(warranty_claims["claim_date"], errors = "coerce")

# fix claim status inconsistency

warranty_claims["claim_status"] = warranty_claims["claim_status"].str.strip().replace({
    "Apprvd" : "Approved"
})

# drop duplicates
warranty_claims.drop_duplicates(inplace = True)

print("Warranty claims table cleaned")
print(warranty_claims.head(), "\n")
print(warranty_claims.info())

Warranty claims table cleaned
  claim_id vehicle_id claim_date  claim_amount claim_status     component
0   W30000     V19829 2023-09-15         11398     Rejected  Transmission
1   W30001     V14602 2024-07-28         10376     Approved  Transmission
2   W30002     V17940 2023-06-20          1980     Rejected    Suspension
3   W30003     V12448 2022-09-23          5738     Approved     Brake Pad
4   W30004     V13902 2023-01-01          6576     Rejected     Brake Pad 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   claim_id      20000 non-null  object        
 1   vehicle_id    20000 non-null  object        
 2   claim_date    20000 non-null  datetime64[ns]
 3   claim_amount  20000 non-null  int64         
 4   claim_status  20000 non-null  object        
 5   component     20000 non-null  object        
dtypes: datetime

# Clean sensor data table

In [25]:
# convert timestamp into datetime dtype

sensor_data["timestamp"] = pd.to_datetime(sensor_data["timestamp"], errors = "coerce")

# convert rpm into numeric dtype

sensor_data["rpm"] = pd.to_numeric(sensor_data["rpm"], errors = "coerce")

# fill missing error codes with "N/a"

sensor_data["error_code"] = sensor_data["error_code"].fillna("N/a")

# fill missing rpm values

sensor_data["rpm"] = sensor_data["rpm"].fillna("N/a")

#drop duplicates
sensor_data.drop_duplicates(inplace = True)

In [31]:
print("Senor data table cleaned")
print(sensor_data.head(), "\n")
print(sensor_data.info())

Senor data table cleaned
  vehicle_id           timestamp  engine_temp     rpm  fuel_efficiency  \
0     V10978 2023-02-12 08:21:00           90  2738.0             18.1   
1     V17423 2023-02-09 00:34:00           72  2991.0             19.4   
2     V17408 2023-02-15 14:55:00           84  1984.0             18.5   
3     V12308 2023-04-20 18:03:00           99  2821.0             10.3   
4     V16018 2023-02-24 22:38:00          108     N/a             14.4   

  error_code  
0       E201  
1       E201  
2        N/a  
3        N/a  
4       E303   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   vehicle_id       100000 non-null  object        
 1   timestamp        100000 non-null  datetime64[ns]
 2   engine_temp      100000 non-null  int64         
 3   rpm              100000 non-null  object        
 4   

# Save cleaned data 


In [35]:
# save vehicles csv 
vehicles.to_csv(os.path.join(clean_path, "vehicles_clean.csv"), index = False)

# save service records csv
service_records.to_csv(os.path.join(clean_path, "service_records_clean.csv"), index = False)

# save warranty claims csv
warranty_claims.to_csv(os.path.join(clean_path, "warranty_claims_clean.csv"), index = False)

# save sensor data csv
sensor_data.to_csv(os.path.join(clean_path, "sensor_data_clean.csv"), index = False)

print("Cleaned data saved to repective path")

Cleaned data saved to repective path
